# agent 返回结构化数据的 几种方式
- ToolStrategy[StructuredResponseT]: Uses tool calling for structured output
- ProviderStrategy[StructuredResponseT]: Uses provider-native structured output | 模型原生输出结构
- type[StructuredResponseT]: Schema type - automatically selects best strategy based on model capabilities
- None: No structured output 

## Provider strategy
可以用 Pydantic model 封装：

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.tools import tools

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model="gpt-5",
    tools=tools,
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

## Tool calling strategy
如果模型不支持原生的 标准结构化输出，可以使用LangChain 的 tool calling 实现同样的 效果。 需要配置一个 ToolStrategy：

````py
class ToolStrategy(Generic[SchemaT]):
    schema: type[SchemaT]
    tool_message_content: str | None
    handle_errors: Union[
        bool,
        str,
        type[Exception],
        tuple[type[Exception], ...],
        Callable[[Exception], str],
    ]
````

schema 参数是必须的，用来定义 结构化输出的 格式：支持：

- ⭐Pydantic models: BaseModel subclasses with field validation
- Dataclasses: Python dataclasses with type annotations
- TypedDict: Typed dictionary classes
- JSON Schema: Dictionary with JSON schema specification
- Union types: Multiple schema options. The model will choose the most appropriate schema based on the context.